In [1]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import panel as pn
import plotly.express as px
import numpy as np
from hvplot import hvPlot
import param
from src.data.make_dataset import DATA_PATH
from src.visualization.visualize import _get_upper_bar, LABELS
from src.visualization.panel_figures import benchmark, profiles
import random

pd.options.plotting.backend = "holoviews"

%load_ext autoreload
%autoreload 2

In [2]:
df = benchmark.get_df()

In [3]:
df.columns

Index(['Id', 'SIREN principal', 'Méthode BEGES (V4,V5)', 'Type de structure',
       'Type de collectivité', 'Mode de consolidation', 'Recalcul',
       'Comparaison avec le précédent bilan', 'nb_salaries_range',
       'nb_salaries_min', 'nb_salaries_max', 'nb_salaries_mean', 'naf5',
       'Secteur d'activité (NAF1)', 'naf2', 'naf3', 'naf4',
       'month_publication', 'Année de reporting', 'Structure obligée',
       'poste_emissions', 'Émissions_totales', 'Émission_par_salarié',
       'emissions_clipped', 'Scope', 'Catégorie d'émissions',
       'Poste d'émissions', 'Année de publication'],
      dtype='object')

## 02 - peut-on comparer ?

In [ ]:
x = benchmark.filter_options(
    df,
    secteur_activite='all',
    #type_structure_all=False, type_structure_options=['Entreprise', 'Établissement public']
)
f = benchmark.plot_n_bilans(x, group_by=LABELS.poste_emissions).opts(
    title="Quels postes d'émissions sont pris en compte dans les bilans GES\ndéposés sur le site de l'ADEME ?"
)
c = pn.Column(
    f,
    pn.pane.Markdown(f"*Tous bilans confondus (n_bilans={x.Id.nunique()})*")#,, align='center'),
)
c.save('totoo.png')

In [ ]:
x = benchmark.filter_options(
    df,
    secteur_activite='all',
    type_structure_all=False, type_structure_options=['Entreprise']
)
print(x.Id.nunique())
benchmark.plot_n_bilans(x, group_by=LABELS.poste_emissions)

In [ ]:
x = benchmark.filter_options(
    df,
    secteur_activite='all',
    type_structure_all=False, type_structure_options=['Entreprise'],
    annee_all=False, annee_options=[2022]
)
print(x.Id.nunique())
benchmark.plot_n_bilans(x, group_by=LABELS.poste_emissions)

## 03 - médiane ou moyenne ?

In [24]:
plot_col = LABELS.emissions_par_salarie
group_by=LABELS.type_structure

x = benchmark.filter_options(
    df,
    secteur_activite='all',
    type_structure_all=False, type_structure_options=['Entreprise', 'Établissement public', 'Association', 'État']
)
x = benchmark.aggregate_bilans(x, plot_col=plot_col, group_by=group_by)
print(x.Id.nunique())
benchmark.plot_emissions(x, plot_col=plot_col, group_by=group_by).opts(
    title="Intensité carbone des bilans GES déposés sur le site de l'ADEME,\npar type de structure",
    ylim = (0, 190),
    width=1000,
)

5180


:Overlay
   .BoxWhisker.I    :BoxWhisker   [Type de structure]   (Émission_par_salarié)
   .Scatter.Moyenne :Scatter   [Type de structure]   (Moyenne)

Look manually what kind of bilans are pushing the average so high

In [42]:
x = x[x[LABELS.type_structure] == 'Entreprise']
x = x.sort_values(LABELS.emissions_par_salarie, ascending=False)
top_20 = x.head(20)
top_20_ids = top_20['Id'].values.tolist()

df_profils = profiles.get_df()
y = pd.merge(top_20, df_profils, on='Id')

In [43]:
y.columns

Index(['Type de structure_x', 'Id', 'Émission_par_salarié',
       'Méthode BEGES (V4,V5)', 'Date de publication', 'Type de structure_y',
       'Type de collectivité', 'Raison sociale', 'SIREN principal',
       'APE(NAF) associé', 'Libellé', 'Nombre de salariés/d'agents',
       'Population', 'Région', 'Code département', 'Département',
       'Structure obligée', 'Mode de consolidation', 'Année de reporting',
       'Assujetti DPEF/PCAET ?', 'Lien DPEF/PCAET',
       'Aide diag décarbon'action', 'Seuil d'importance retenu (%)',
       'Niveau d'influence', 'Importance stratégique et vulnérabilités',
       'Lignes directrices spécifiques au secteur', 'Sous-traitance',
       'Engagement du personnel',
       'Justification des postes d'émissions indirectes écartés',
       'Une année de référence a été calculée',
       'Présentation de l'organisation', 'Politique Développement Durable',
       'Objectif émissions directes',
       'Objectif émissions indirectes significatives', 'Ob

Examen
* 804153468, A Votre Bonheur : valeurs absurdes, 200 000 tCO2 eq, pour 3-5 employés ??
* 404117400, JIMENEZ TRANSPORT & LOCATION : `28 537,996.00` interprété comme `28537996` -> problème de virgule
* 061500245, CATERPILLAR FRANCE SAS : problème de virgule aussi
* 055801849, CERESINE SASU : valeurs absurdes, 997 282 tCO2 pour 20-49 employés ??
* 562087346, BONNA SABLA : 1 valeur absurde, qu'on retrouve dans l'export mais pas sur le site bilans-ges (corrigé?)

In [44]:
y[[
    'Émission_par_salarié',
       'Méthode BEGES (V4,V5)', 'Date de publication',
    'Raison sociale', 'SIREN principal',
    'Libellé', "Nombre de salariés/d'agents",
    'Structure obligée', 'Mode de consolidation', 'Année de reporting',
    'Incertitudes'
]]

,Émission_par_salarié,"Méthode BEGES (V4,V5)",Date de publication,Raison sociale,SIREN principal,Libellé,Nombre de salariés/d'agents,Structure obligée,Mode de consolidation,Année de reporting,Incertitudes
0,49937.000000,v4,28/09/2019,A Votre Bonheur,804153468,Aide à domicile,Entre 3 et 5,non,Opérationnel,2018,NaN
1,38083.647765,v4,30/03/2021,JIMENEZ TRANSPORT & LOCATION,404117400,Transports routiers de fret de proximité,Entre 500 et 999,oui,Opérationnel,2020,NaN
2,29235.551184,v4,20/07/2021,CATERPILLAR FRANCE SAS,061500245,Fabrication de machines pour l'extraction ou l...,Entre 1 000 et 1 999,oui,Opérationnel,2018,La personne morale doit présenter des éléments...
3,28906.724638,v4,16/11/2022,CERESINE SASU,055801849,"Fabrication de savons, détergents et produits ...",Entre 20 et 49,non,Opérationnel,2022,NaN
4,21614.848236,v4,21/11/2022,BONNA SABLA,562087346,Activités des sièges sociaux,Entre 500 et 999,oui,Opérationnel,2021,NaN
5,19879.172414,v4,08/08/2022,SECURI TP,422854919,"Analyses, essais et inspections techniques",Entre 10 et 19,non,Opérationnel,2021,\n
6,17466.666667,v4,28/12/2022,SOCIETE ANONYME COOPERATIVE D'INTERET COLLECTI...,005880679,Promotion immobilière de logements,Entre 1 et 2,non,Opérationnel,2021,NaN
7,15281.510368,v4,14/11/2022,DNCA,432518041,Gestion de fonds,Entre 100 et 199,oui,Opérationnel,2021,NaN
8,13890.324216,v4,22/12/2022,REVIVAL,616620092,Récupération de déchets triés,Entre 500 et 999,oui,Opérationnel,2021,4% incertidue pour le Gaz (outil Bilan Carbone...
9,13665.216811,v4,06/04/2016,Pompes GRUNDFOS France,372800227,Fabrication d'autres pompes et compresseurs,Entre 500 et 999,oui,Opérationnel,2014,Les incertitudes sur les facteurs d’émissions ...


## 01 - scope 3

In [4]:
benchmark.get_benchmark_dashboard()

FlexBox(objects=[Column(max_width=350, ...], sizing_mode='stretch_width')

## debug 2

In [5]:
plot_col = LABELS.emissions_par_salarie
group_by=LABELS.secteur_activite

x = benchmark.filter_options(
    df,
    secteur_activite='all'
)
x = benchmark.aggregate_bilans(x, plot_col=plot_col, group_by=group_by)

In [7]:
df[LABELS.secteur_activite].str.replace('nan', 'nannnn').unique()

array(["Activités finannnncières et d'assurance",
       "Commerce ; réparation d'automobiles et de motocycles",
       'Santé humaine et action sociale', 'Administration publique',
       'Activités immobilières', 'nannnn', 'Autres activités de services',
       "Production et distribution d'eau ; assainissement, gestion des déchets et dépollution",
       'Information et communication', 'Industrie manufacturière',
       'Activités de services administratifs et de soutien',
       'Activités spécialisées, scientifiques et techniques',
       'Transports et entreposage', 'Enseignement',
       "Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné",
       'Activités extra-territoriales', 'Hébergement et restauration',
       'Construction', 'Agriculture, sylviculture et pêche',
       'Arts, spectacles et activités récréatives',
       'Industries extractives'], dtype=object)

In [9]:
x[LABELS.secteur_activite].unique()

array([0, 'Activités de services administratifs et de soutien',
       'Activités extra-territoriales',
       "Activités financières et d'assurance", 'Activités immobilières',
       'Activités spécialisées, scientifiques et techniques',
       'Administration publique', 'Agriculture, sylviculture et pêche',
       'Arts, spectacles et activités récréatives',
       'Autres activités de services',
       "Commerce ; réparation d'automobiles et de motocycles",
       'Construction', 'Enseignement', 'Hébergement et restauration',
       'Industrie manufacturière', 'Industries extractives',
       'Information et communication',
       "Production et distribution d'eau ; assainissement, gestion des déchets et dépollution",
       "Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné",
       'Santé humaine et action sociale', 'Transports et entreposage'],
      dtype=object)

### investigate

In [5]:
df.groupby(LABELS.poste_emissions)[LABELS.emissions_par_salarie].mean()

Poste d'émissions
1.1 - Émissions directes des sources fixes de combustion                                28.811056
1.2 - Émissions directes des sources mobiles de combustion                              17.145287
1.3 - Émissions directes des procédés hors énergie                                      10.866147
1.4 - Émissions directes fugitives                                                       0.885532
1.5 - Émissions issues de la biomasse (sols et forêts)                                   0.183633
2.1 - Émissions indirectes liées à la consommation d'électricité                         4.216190
2.2 - Émissions indirectes liées à la consommation d'énergie autre que l'électricité     1.453683
3.1 - Transport de marchandise amont                                                    15.604182
3.2 - Transport de marchandise aval                                                      7.256725
3.3 - Déplacements domicile-travail                                                     19.810089
3.

In [6]:
plot_col = LABELS.emissions_par_salarie
group_by=LABELS.poste_emissions

x = benchmark.filter_options(
    df,
    secteur_activite='all'
)
#x = benchmark.aggregate_bilans(x, plot_col=plot_col, group_by=group_by)

In [7]:
print(x[LABELS.emissions_par_salarie].isna().sum())
print(x[LABELS.emissions_par_salarie].eq(0).sum())
x[[LABELS.poste_emissions, 'Id'] + benchmark.PLOT_COL_OPTIONS].sort_values([LABELS.poste_emissions, 'Id'])

0
0


,Poste d'émissions,Id,Émission_par_salarié,Émissions_totales
5337,1.1 - Émissions directes des sources fixes de ...,00467d43-41a9-41de-8df4-6d9260bdb8b7,0.544603,408.18
5163,1.1 - Émissions directes des sources fixes de ...,00609c55-d063-4661-9c22-b161bb72bc99,0.007738,5.80
5321,1.1 - Émissions directes des sources fixes de ...,00aef8a9-88ed-4c21-bba3-b03af292ef60,0.063549,47.63
5149,1.1 - Émissions directes des sources fixes de ...,015b4696-b05d-45ce-bfdd-2564326412c5,0.712447,5343.00
5137,1.1 - Émissions directes des sources fixes de ...,0165ace2-634c-458d-9b66-52578a59ab86,15.210702,2274.00
...,...,...,...,...
123968,6.1 - Autres émissions directes,fb64803d-142a-4915-9ee7-3a2be2f3bcc9,0.136091,102.00
123737,6.1 - Autres émissions directes,fd0efaba-5989-46c1-a8f4-a7d0b5a980ed,0.619079,464.00
123675,6.1 - Autres émissions directes,fd625530-0a76-4e91-b89f-4560a427ff10,0.044578,156.00
123645,6.1 - Autres émissions directes,fd97e2bb-35f5-4fcc-9253-7622f5651f34,0.091275,6.80


In [8]:
124058 - (25552 + 62194)

36312

In [9]:
x.groupby(LABELS.poste_emissions)[LABELS.emissions_par_salarie].mean()

Poste d'émissions
1.1 - Émissions directes des sources fixes de combustion                                 31.324772
1.2 - Émissions directes des sources mobiles de combustion                               18.296348
1.3 - Émissions directes des procédés hors énergie                                       45.178148
1.4 - Émissions directes fugitives                                                        1.189220
1.5 - Émissions issues de la biomasse (sols et forêts)                                    2.998985
2.1 - Émissions indirectes liées à la consommation d'électricité                          4.263779
2.2 - Émissions indirectes liées à la consommation d'énergie autre que l'électricité      3.671049
3.1 - Transport de marchandise amont                                                     30.796955
3.2 - Transport de marchandise aval                                                      24.283151
3.3 - Déplacements domicile-travail                                                      25

In [10]:
y = benchmark.aggregate_bilans(x, plot_col=plot_col, group_by=group_by)
print(y[LABELS.emissions_par_salarie].isna().sum())
print(y[LABELS.emissions_par_salarie].eq(0).sum())
y[[LABELS.poste_emissions, 'Id'] + benchmark.PLOT_COL_OPTIONS].sort_values([LABELS.poste_emissions, 'Id'])

0
0


,Poste d'émissions,Id,Émission_par_salarié,Émissions_totales
0,1.1 - Émissions directes des sources fixes de ...,00467d43-41a9-41de-8df4-6d9260bdb8b7,0.544603,408.18
1,1.1 - Émissions directes des sources fixes de ...,00609c55-d063-4661-9c22-b161bb72bc99,0.007738,5.80
2,1.1 - Émissions directes des sources fixes de ...,00aef8a9-88ed-4c21-bba3-b03af292ef60,0.063549,47.63
3,1.1 - Émissions directes des sources fixes de ...,015b4696-b05d-45ce-bfdd-2564326412c5,0.712447,5343.00
4,1.1 - Émissions directes des sources fixes de ...,0165ace2-634c-458d-9b66-52578a59ab86,15.210702,2274.00
...,...,...,...,...
36307,6.1 - Autres émissions directes,fb64803d-142a-4915-9ee7-3a2be2f3bcc9,0.136091,102.00
36308,6.1 - Autres émissions directes,fd0efaba-5989-46c1-a8f4-a7d0b5a980ed,0.619079,464.00
36309,6.1 - Autres émissions directes,fd625530-0a76-4e91-b89f-4560a427ff10,0.044578,156.00
36310,6.1 - Autres émissions directes,fd97e2bb-35f5-4fcc-9253-7622f5651f34,0.091275,6.80


In [14]:
y = benchmark.aggregate_bilans(x, plot_col=plot_col, group_by=LABELS.category_emissions)
print(y[LABELS.emissions_par_salarie].isna().sum())
print(y[LABELS.emissions_par_salarie].eq(0).sum())
y[[LABELS.category_emissions, 'Id'] + benchmark.PLOT_COL_OPTIONS].sort_values([LABELS.category_emissions, 'Id'])

0
0


,Catégorie d'émissions,Id,Émission_par_salarié,Émissions_totales
0,1 - Émissions directes,00467d43-41a9-41de-8df4-6d9260bdb8b7,1.155757,866.24
1,1 - Émissions directes,00609c55-d063-4661-9c22-b161bb72bc99,0.290887,218.02
2,1 - Émissions directes,00aef8a9-88ed-4c21-bba3-b03af292ef60,0.762855,571.76
3,1 - Émissions directes,015b4696-b05d-45ce-bfdd-2564326412c5,0.746716,5600.00
4,1 - Émissions directes,0165ace2-634c-458d-9b66-52578a59ab86,24.093645,3602.00
...,...,...,...,...
18120,6 - Autres émissions indirectes,fb64803d-142a-4915-9ee7-3a2be2f3bcc9,0.136091,102.00
18121,6 - Autres émissions indirectes,fd0efaba-5989-46c1-a8f4-a7d0b5a980ed,0.619079,464.00
18122,6 - Autres émissions indirectes,fd625530-0a76-4e91-b89f-4560a427ff10,0.044578,156.00
18123,6 - Autres émissions indirectes,fd97e2bb-35f5-4fcc-9253-7622f5651f34,0.091275,6.80


In [16]:
y.groupby(LABELS.category_emissions)[LABELS.emissions_par_salarie].mean()

Catégorie d'émissions
1 - Émissions directes              51.558191
2 - Énergie                          5.217670
3 - Déplacement                     51.640281
4 - Produits achetés                60.258308
5 - Produits vendus                197.368282
6 - Autres émissions indirectes     20.130501
Name: Émission_par_salarié, dtype: float64

## debug venv issue

In [8]:
df = profiles.get_df()

In [11]:
from src.visualization.panel_figures.profiles import LABELS, _LABELS_NUNIQUE

In [41]:
x = df.rename(columns=_LABELS_NUNIQUE)

# Compute the reporting year relative to publication year
x[LABELS.annee_reporting_rel] = (
    x["Année de reporting"] - x[LABELS.annee_publication]
)

# Get a nice label for that
def label_n(n):
    if n == 0:
        return "N"
    if n > 0:
        # strangely, some publications report data for years in the future
        return "> N"
    if n <= -5:
        # some go far in the past (N-11), regroup everything to be less spammy
        return "< N-5"
    return f"N-{abs(n)}"

df[LABELS.annee_reporting_rel] = (
    df["Année de reporting"] - df[LABELS.annee_publication]
)
x[LABELS.annee_reporting_rel_label] = x[LABELS.annee_reporting_rel].map(label_n)

x = x[x[LABELS.annee_publication] > 2020]

# build a custom colormap: the last value is red on purpose, to indicate
# that '> N' is not a normal value
n_labels = x[LABELS.annee_reporting_rel_label].nunique()

x[LABELS.annee_publication] = x[LABELS.annee_publication].astype(str)

x = x.sort_values([LABELS.annee_publication, LABELS.annee_reporting_rel])
x = x.groupby(
    [LABELS.annee_publication, LABELS.annee_reporting_rel_label], sort=False
)
x = x[LABELS.n_bilans].nunique()

In [42]:
x.to_dict()

{('2021', '< N-5'): 18,
 ('2021', 'N-4'): 21,
 ('2021', 'N-3'): 59,
 ('2021', 'N-2'): 278,
 ('2021', 'N-1'): 301,
 ('2021', 'N'): 19,
 ('2022', '< N-5'): 23,
 ('2022', 'N-4'): 42,
 ('2022', 'N-3'): 207,
 ('2022', 'N-2'): 172,
 ('2022', 'N-1'): 699,
 ('2022', 'N'): 51,
 ('2023', '< N-5'): 6,
 ('2023', 'N-4'): 49,
 ('2023', 'N-3'): 27,
 ('2023', 'N-2'): 216,
 ('2023', 'N-1'): 299,
 ('2023', 'N'): 4}

In [43]:
x.plot(kind="bar", stacked=True).opts()

object


:Bars   [Année de publication,Année de reporting (relative)]   (Nb. bilans déposés)

In [10]:
profiles.plot_annee_publication(df)

/Users/alexandre/dev/carbon-trackr/benchmark-footprints/venv/lib/python3.11/site-packages/holoviews/plotting/bokeh/chart.py:930: RuntimeWarning:

divide by zero encountered in divide



:Bars   [Année de publication,Année de reporting (relative)]   (Nb. bilans déposés)